# 전이 학습 후 파인튜닝 없이 분류기 부분만 훈련

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from PIL import Image
import os
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import random

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

from tqdm import tqdm

In [2]:
v = '1'
epochs = 11

In [3]:
# 완벽한 실험 재현성을 위한 랜덤제어
random_seed = 28
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU

In [4]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060


In [5]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.Resize(256),
        transforms.CenterCrop(224),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),    
}

In [6]:
image_path = "C:/Users/ANDlab3/Desktop/final train/data/vision_data/"
image_datasets = {x: datasets.ImageFolder(os.path.join(image_path, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val','test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [7]:
dataset_sizes

{'train': 6475, 'val': 925, 'test': 1850}

In [8]:
class_num = len(class_names)
class_num

37

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
pre_model = models.resnet50(pretrained=True)
num_ftrs = pre_model.fc.in_features

#Changing the number of outputs in the last layer to the number of different item types
pre_model.fc = nn.Linear(num_ftrs, 37)

In [11]:
# class vision_module(nn.Module):
#     def __init__(self, pre_model):
#         super(vision_module, self).__init__()
    
#         D_in, H, D_out = 1000, 500, 37
#         self.resnet50 = pre_model
        
#         self.classifier = nn.Sequential(
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(H, D_out)
#         )
    
#     def forward(self, image):
#         outputs = self.resnet50(image)
        
#         fc = self.classifier(outputs)
        
#         return  fc

In [12]:
model_ft = pre_model.to(device)
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# model_ft = vision_module(pre_model)
# model_ft.to(device)

In [14]:
# 파라메타 번호 확인 하기
i = 0
for name, param in model_ft.named_parameters():
    
    print(i,name)
    i+= 1

0 conv1.weight
1 bn1.weight
2 bn1.bias
3 layer1.0.conv1.weight
4 layer1.0.bn1.weight
5 layer1.0.bn1.bias
6 layer1.0.conv2.weight
7 layer1.0.bn2.weight
8 layer1.0.bn2.bias
9 layer1.0.conv3.weight
10 layer1.0.bn3.weight
11 layer1.0.bn3.bias
12 layer1.0.downsample.0.weight
13 layer1.0.downsample.1.weight
14 layer1.0.downsample.1.bias
15 layer1.1.conv1.weight
16 layer1.1.bn1.weight
17 layer1.1.bn1.bias
18 layer1.1.conv2.weight
19 layer1.1.bn2.weight
20 layer1.1.bn2.bias
21 layer1.1.conv3.weight
22 layer1.1.bn3.weight
23 layer1.1.bn3.bias
24 layer1.2.conv1.weight
25 layer1.2.bn1.weight
26 layer1.2.bn1.bias
27 layer1.2.conv2.weight
28 layer1.2.bn2.weight
29 layer1.2.bn2.bias
30 layer1.2.conv3.weight
31 layer1.2.bn3.weight
32 layer1.2.bn3.bias
33 layer2.0.conv1.weight
34 layer2.0.bn1.weight
35 layer2.0.bn1.bias
36 layer2.0.conv2.weight
37 layer2.0.bn2.weight
38 layer2.0.bn2.bias
39 layer2.0.conv3.weight
40 layer2.0.bn3.weight
41 layer2.0.bn3.bias
42 layer2.0.downsample.0.weight
43 layer2.0.do

In [15]:
for i, (name, param) in enumerate(model_ft.named_parameters()):
    
    param.requires_grad = False
    if i == 158:
        print('end')
        break

end


In [16]:
for p in model_ft.named_parameters():
    print(p)
#     break

('conv1.weight', Parameter containing:
tensor([[[[ 1.3335e-02,  1.4664e-02, -1.5351e-02,  ..., -4.0896e-02,
           -4.3034e-02, -7.0755e-02],
          [ 4.1205e-03,  5.8477e-03,  1.4948e-02,  ...,  2.2060e-03,
           -2.0912e-02, -3.8517e-02],
          [ 2.2331e-02,  2.3595e-02,  1.6120e-02,  ...,  1.0281e-01,
            6.2641e-02,  5.1977e-02],
          ...,
          [-9.0349e-04,  2.7767e-02, -1.0105e-02,  ..., -1.2722e-01,
           -7.6604e-02,  7.8453e-03],
          [ 3.5894e-03,  4.8006e-02,  6.2051e-02,  ...,  2.4267e-02,
           -3.3662e-02, -1.5709e-02],
          [-8.0029e-02, -3.2238e-02, -1.7808e-02,  ...,  3.5359e-02,
            2.2439e-02,  1.7077e-03]],

         [[-1.8452e-02,  1.1415e-02,  2.3850e-02,  ...,  5.3736e-02,
            4.4022e-02, -9.4675e-03],
          [-7.7273e-03,  1.8890e-02,  6.7981e-02,  ...,  1.5956e-01,
            1.4606e-01,  1.1999e-01],
          [-4.6013e-02, -7.6075e-02, -8.9648e-02,  ...,  1.2108e-01,
            1.6705e

('layer1.1.conv1.weight', Parameter containing:
tensor([[[[ 0.0250]],

         [[ 0.0026]],

         [[ 0.0110]],

         ...,

         [[ 0.0004]],

         [[ 0.0207]],

         [[-0.0345]]],


        [[[ 0.0023]],

         [[-0.0012]],

         [[-0.0111]],

         ...,

         [[-0.0166]],

         [[ 0.0230]],

         [[-0.0736]]],


        [[[-0.0060]],

         [[-0.0019]],

         [[ 0.0085]],

         ...,

         [[-0.0060]],

         [[-0.0386]],

         [[ 0.0282]]],


        ...,


        [[[ 0.0059]],

         [[ 0.0186]],

         [[ 0.0013]],

         ...,

         [[ 0.0106]],

         [[-0.0156]],

         [[-0.0496]]],


        [[[ 0.0453]],

         [[-0.0053]],

         [[-0.0131]],

         ...,

         [[ 0.0219]],

         [[ 0.0367]],

         [[-0.0212]]],


        [[[ 0.0066]],

         [[ 0.0191]],

         [[ 0.0143]],

         ...,

         [[-0.0037]],

         [[-0.0251]],

         [[ 0.0725]]]], device='

('layer1.2.bn2.weight', Parameter containing:
tensor([0.2164, 0.2510, 0.2231, 0.2292, 0.2206, 0.2179, 0.2162, 0.1634, 0.2093,
        0.2222, 0.2535, 0.3154, 0.2342, 0.2426, 0.2290, 0.2137, 0.2216, 0.1808,
        0.2402, 0.2401, 0.2296, 0.2416, 0.2316, 0.2086, 0.2435, 0.2377, 0.2285,
        0.2185, 0.2242, 0.2128, 0.2328, 0.2420, 0.1663, 0.2489, 0.1979, 0.2088,
        0.2402, 0.1934, 0.1159, 0.2452, 0.2377, 0.1765, 0.2250, 0.1855, 0.2238,
        0.2285, 0.2402, 0.2207, 0.2157, 0.2389, 0.2259, 0.2190, 0.2020, 0.2269,
        0.2276, 0.2248, 0.2533, 0.1996, 0.2053, 0.1992, 0.2117, 0.2513, 0.2136,
        0.2317], device='cuda:0'))
('layer1.2.bn2.bias', Parameter containing:
tensor([-0.0306, -0.0923, -0.0414, -0.0653, -0.1028, -0.0986, -0.0136, -0.0025,
        -0.1152, -0.0291, -0.0657, -0.3410, -0.0687, -0.0501, -0.0541, -0.0482,
        -0.0685,  0.0415, -0.0591, -0.0672, -0.0274, -0.1126, -0.0099, -0.0306,
        -0.1327, -0.0547, -0.0480, -0.0265, -0.0448, -0.0258, -0.0761, -0.1

('layer2.0.bn3.weight', Parameter containing:
tensor([ 3.8543e-03,  2.0187e-01, -1.0649e-03,  1.3887e-01,  6.4771e-03,
        -2.0702e-06,  2.0280e-01,  1.1607e-02, -4.1076e-03,  2.7771e-02,
         8.9216e-02,  1.0155e-01,  1.0902e-01,  3.9999e-02,  4.5550e-06,
         1.2153e-01,  2.9691e-01, -1.7842e-02,  1.3224e-01,  9.2594e-02,
         8.6137e-02,  2.3588e-03,  2.1034e-01,  1.5372e-07,  1.6275e-01,
        -1.0454e-02,  1.1453e-01,  1.5497e-01,  6.6282e-06,  1.6580e-01,
         3.1317e-01, -2.4331e-03,  5.2280e-02,  1.5141e-01,  8.9488e-02,
         1.9418e-01,  7.4374e-02,  1.9285e-01,  2.3015e-03,  2.2057e-01,
         2.4289e-01,  1.8490e-01,  6.0817e-02,  2.4235e-03, -7.2726e-03,
         2.1065e-01,  1.1995e-02,  1.9941e-01,  1.5138e-01,  2.0406e-01,
        -3.2079e-03,  1.2165e-04,  6.7944e-02,  9.9864e-02,  2.7700e-03,
         2.0632e-01,  4.6338e-02,  8.8209e-02,  2.0853e-01, -7.7076e-08,
        -1.2983e-03,  1.4655e-01,  1.3156e-01,  6.1504e-03,  1.5142e-01,
     

('layer2.0.downsample.0.weight', Parameter containing:
tensor([[[[ 6.9636e-02]],

         [[ 3.6008e-03]],

         [[ 1.3478e-02]],

         ...,

         [[ 1.8134e-01]],

         [[ 1.9458e-03]],

         [[ 3.1590e-03]]],


        [[[-1.0936e-02]],

         [[-3.4424e-03]],

         [[ 1.6514e-02]],

         ...,

         [[ 3.2527e-03]],

         [[-3.9140e-02]],

         [[-5.9866e-03]]],


        [[[ 8.8925e-04]],

         [[ 1.0271e-03]],

         [[ 1.5778e-03]],

         ...,

         [[ 2.5194e-04]],

         [[-2.1439e-03]],

         [[-1.1421e-04]]],


        ...,


        [[[ 1.0406e-07]],

         [[ 5.3282e-08]],

         [[ 9.3874e-08]],

         ...,

         [[-3.8367e-08]],

         [[-1.5700e-07]],

         [[ 1.0596e-07]]],


        [[[ 8.8278e-03]],

         [[-2.4494e-03]],

         [[-4.0442e-02]],

         ...,

         [[ 1.2038e-02]],

         [[ 2.7628e-02]],

         [[ 1.4599e-02]]],


        [[[ 7.5303e-03]],

        

...

('layer3.4.bn1.bias', Parameter containing:
tensor([ 0.0006, -0.0725, -0.0153, -0.0658, -0.0610, -0.2592, -0.1810, -0.1490,
        -0.0697, -0.0946, -0.1224, -0.1609,  0.0562, -0.0820, -0.1319, -0.1549,
        -0.1721, -0.0335, -0.0094, -0.1652, -0.1359, -0.1245, -0.0630, -0.0685,
        -0.1001, -0.0173, -0.1559, -0.1228, -0.0454, -0.1486, -0.1025, -0.0473,
        -0.1797, -0.1243, -0.0262, -0.0965, -0.0025,  0.0196, -0.0636, -0.0974,
        -0.2064, -0.0293, -0.1324, -0.1492, -0.1018, -0.1239,  0.0444, -0.0776,
        -0.1004, -0.1010, -0.0011, -0.0309, -0.0339, -0.1574, -0.0913, -0.1674,
        -0.0259, -0.0579, -0.0208, -0.0215, -0.1362,  0.0058, -0.0442, -0.1441,
        -0.0608, -0.0837,  0.0099,  0.0036, -0.1816, -0.0252, -0.1104, -0.0940,
        -0.0362, -0.0573, -0.1263, -0.0292, -0.1641, -0.0536,  0.0529, -0.0321,
        -0.0257, -0.0947, -0.1793, -0.1640, -0.0930, -0.1015, -0.1547, -0.1717,
         0.0330, -0.1644, -0.0914, -0.1588, -0.1597,  0.0845, -0.0404, -0.27

('layer3.5.bn2.bias', Parameter containing:
tensor([-0.0950, -0.0429, -0.0648, -0.1112, -0.1368, -0.0984, -0.1285, -0.0326,
        -0.1890, -0.0512, -0.1933, -0.0564, -0.1662, -0.1181, -0.1060, -0.0185,
        -0.0933, -0.0582, -0.0318, -0.1021, -0.1560, -0.0770, -0.1286, -0.0659,
        -0.1850, -0.0679, -0.0762, -0.0659, -0.0736, -0.0134, -0.1070, -0.0684,
        -0.0896, -0.0029, -0.1198, -0.0119, -0.1063, -0.0686, -0.0612, -0.1319,
        -0.1781, -0.0765,  0.1662, -0.1244, -0.0856, -0.0270, -0.1248, -0.0530,
        -0.2007, -0.0635,  0.1194, -0.1497, -0.0671, -0.1742, -0.1035, -0.1167,
        -0.1681, -0.1027, -0.1438, -0.0405,  0.0730, -0.0770, -0.0578, -0.0834,
        -0.2014, -0.1179, -0.0407, -0.0703,  0.0021, -0.0678, -0.1495, -0.0342,
        -0.0983, -0.1059, -0.0215, -0.0554, -0.0783, -0.0742, -0.0601, -0.0918,
        -0.0507, -0.1486, -0.0749, -0.0901, -0.0635, -0.0740, -0.0870, -0.0976,
        -0.2599, -0.0890, -0.0899, -0.1529, -0.0424, -0.1403, -0.1159, -0.05

('layer4.0.bn2.weight', Parameter containing:
tensor([0.2305, 0.2149, 0.2113, 0.2504, 0.1993, 0.2356, 0.2335, 0.2072, 0.2152,
        0.2138, 0.1926, 0.1850, 0.2060, 0.2176, 0.2039, 0.2212, 0.1863, 0.1873,
        0.1493, 0.1916, 0.1705, 0.2303, 0.1862, 0.1900, 0.1700, 0.2087, 0.1958,
        0.2270, 0.1818, 0.1843, 0.1867, 0.1674, 0.1857, 0.1507, 0.2051, 0.2016,
        0.1635, 0.2090, 0.2137, 0.1697, 0.1658, 0.1667, 0.1646, 0.1953, 0.1907,
        0.1708, 0.1973, 0.1910, 0.2409, 0.2094, 0.1890, 0.1803, 0.1537, 0.2171,
        0.2022, 0.2190, 0.2084, 0.2063, 0.1961, 0.2263, 0.2096, 0.1797, 0.1928,
        0.2004, 0.2098, 0.1724, 0.1908, 0.1635, 0.2154, 0.1940, 0.1532, 0.2299,
        0.2036, 0.1993, 0.1817, 0.1758, 0.1739, 0.1898, 0.1723, 0.1854, 0.2060,
        0.2025, 0.2072, 0.1955, 0.1988, 0.1879, 0.2443, 0.1874, 0.1690, 0.2082,
        0.2248, 0.2016, 0.2220, 0.2259, 0.2958, 0.2071, 0.1808, 0.2133, 0.1863,
        0.1748, 0.2262, 0.1895, 0.1894, 0.2384, 0.2254, 0.1897, 0.2026, 0.

('layer4.1.bn2.weight', Parameter containing:
tensor([0.1877, 0.2265, 0.1861, 0.2233, 0.2291, 0.2273, 0.1859, 0.2056, 0.2021,
        0.2021, 0.2152, 0.2187, 0.1999, 0.2125, 0.2047, 0.1989, 0.2277, 0.2308,
        0.2018, 0.1943, 0.1823, 0.2081, 0.2040, 0.2022, 0.2179, 0.1940, 0.1963,
        0.2029, 0.2220, 0.2233, 0.2097, 0.2119, 0.2199, 0.1800, 0.2060, 0.1578,
        0.1966, 0.1982, 0.1960, 0.2210, 0.2283, 0.1959, 0.2060, 0.1961, 0.2044,
        0.1890, 0.2079, 0.2204, 0.2132, 0.2264, 0.2237, 0.1906, 0.1848, 0.2075,
        0.2095, 0.1922, 0.1978, 0.2317, 0.1926, 0.2640, 0.2141, 0.2147, 0.2118,
        0.1930, 0.1537, 0.2294, 0.2204, 0.2024, 0.2028, 0.2018, 0.2172, 0.2046,
        0.2046, 0.2256, 0.1912, 0.2290, 0.1933, 0.1819, 0.2002, 0.2029, 0.2438,
        0.2132, 0.2061, 0.2058, 0.2035, 0.2016, 0.2155, 0.1986, 0.2122, 0.2041,
        0.2133, 0.2339, 0.2279, 0.1976, 0.2176, 0.1965, 0.2141, 0.2003, 0.2176,
        0.1982, 0.2055, 0.2103, 0.1931, 0.2011, 0.1790, 0.1800, 0.2139, 0.

('layer4.2.conv2.weight', Parameter containing:
tensor([[[[ 4.0117e-03,  6.4560e-03,  7.5797e-03],
          [ 6.3191e-03,  8.6818e-03,  1.0155e-02],
          [ 1.0467e-02,  7.4264e-03,  1.0289e-02]],

         [[ 8.9806e-03,  8.8232e-03,  1.2227e-02],
          [ 7.9335e-03,  1.2900e-02,  5.7527e-03],
          [ 7.0791e-03,  1.0065e-02,  4.1151e-03]],

         [[ 6.3044e-03,  5.0667e-03,  5.7373e-03],
          [-2.9706e-04, -2.2638e-03,  7.9506e-05],
          [ 6.0903e-04,  3.4252e-04,  2.0867e-03]],

         ...,

         [[-4.3498e-03,  6.7560e-03,  3.5414e-03],
          [ 4.0447e-03,  1.1341e-02,  1.0289e-02],
          [ 6.2327e-03,  1.8316e-02,  1.1889e-02]],

         [[-4.3072e-04, -5.9304e-04, -2.7477e-03],
          [ 1.7383e-03,  1.1411e-03, -2.5095e-03],
          [-7.0297e-03, -8.4976e-03, -1.1197e-02]],

         [[-6.5198e-03, -9.3173e-03, -6.2551e-03],
          [-3.1477e-04,  1.2694e-03,  5.0760e-03],
          [-1.5420e-03, -1.1563e-03,  1.1397e-03]]],


     

In [17]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [18]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)

Epoch 0/10
----------
train Loss: 1.1775 Acc: 0.6724
val Loss: 0.5057 Acc: 0.8259

Epoch 1/10
----------
train Loss: 0.5809 Acc: 0.8088
val Loss: 0.4687 Acc: 0.8454

Epoch 2/10
----------
train Loss: 0.4925 Acc: 0.8366
val Loss: 0.4272 Acc: 0.8508

Epoch 3/10
----------
train Loss: 0.4474 Acc: 0.8510
val Loss: 0.4905 Acc: 0.8346

Epoch 4/10
----------
train Loss: 0.4119 Acc: 0.8625
val Loss: 0.4275 Acc: 0.8519

Epoch 5/10
----------
train Loss: 0.3806 Acc: 0.8701
val Loss: 0.3996 Acc: 0.8649

Epoch 6/10
----------
train Loss: 0.3527 Acc: 0.8820
val Loss: 0.4458 Acc: 0.8562

Epoch 7/10
----------
train Loss: 0.2473 Acc: 0.9181
val Loss: 0.3922 Acc: 0.8714

Epoch 8/10
----------
train Loss: 0.2378 Acc: 0.9198
val Loss: 0.3974 Acc: 0.8659

Epoch 9/10
----------
train Loss: 0.2398 Acc: 0.9208
val Loss: 0.3909 Acc: 0.8659

Epoch 10/10
----------
train Loss: 0.2308 Acc: 0.9228
val Loss: 0.3830 Acc: 0.8746

Training complete in 7m 19s
Best val Acc: 0.874595


In [19]:
SAVE_PATH = "C:/Users/ANDlab3/Desktop/final train/model/vision_model/"
torch.save(model_ft.state_dict(), SAVE_PATH + 'vision_model_fine_tuned'+v+'.pt')

# Test

In [20]:
correct = 0
total = 0

true = []
pred = []

with torch.no_grad():
    model_ft.eval() #현재는 모델에 드랍아웃이나, 패딩이 없어서 필요없지만 만약 사용된 경우에는 eval을 통해서 평가시에는 꼭 비활성화 시켜야한다.
    f1_score = 0
    for data in dataloaders['test']:
        images, labels = data[0].to(device), data[1].to(device)
        
        outputs =  model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        
        
        total += labels.size(0) # 개수 누적(총 개수)
        correct += (predicted == labels).sum().item() # 누적(맞으면 1, 틀리면 0으로 합산)
  
        true += labels.cpu().numpy().tolist()
        pred += predicted.cpu().numpy().tolist()

print('done')

done


In [21]:
accuracy = accuracy_score(true, pred)
round((accuracy * 100), 4)

87.027

In [22]:
print('전체 데이터 수 : ', total)
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

전체 데이터 수 :  1850
Accuracy of the network on the 10000 test images: 87 %


In [23]:
# Classification Report 저장
REPORT_PATH = "C:/Users/ANDlab3/Desktop/final train/report/vision_report/"
CL_REPORT_FILE = REPORT_PATH + "cl_report_"+v+".csv"

cl_report = classification_report(true, pred, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(4)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

              precision  recall  f1-score    support
0                1.0000  0.9600    0.9796    50.0000
1                1.0000  0.9800    0.9899    50.0000
2                1.0000  1.0000    1.0000    50.0000
3                0.9796  0.9600    0.9697    50.0000
4                0.9800  0.9800    0.9800    50.0000
5                0.6842  0.5200    0.5909    50.0000
6                0.8200  0.8200    0.8200    50.0000
7                0.9057  0.9600    0.9320    50.0000
8                0.9070  0.7800    0.8387    50.0000
9                0.9434  1.0000    0.9709    50.0000
10               0.5000  0.3600    0.4186    50.0000
11               0.8776  0.8600    0.8687    50.0000
12               0.7538  0.9800    0.8522    50.0000
13               0.9574  0.9000    0.9278    50.0000
14               0.5714  0.5600    0.5657    50.0000
15               0.6909  0.7600    0.7238    50.0000
16               0.9000  0.9000    0.9000    50.0000
17               0.8070  0.9200    0.8598    5